In [1]:

import nltk
from nltk.stem.lancaster import LancasterStemmer
from nltk.tokenize import sent_tokenize
stemmer = LancasterStemmer()

import numpy as np
import tflearn
import tensorflow as tf
import random

In [2]:
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)
    

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        
        #p = pattern.encode("utf-8")
        w = nltk.word_tokenize(pattern)
        #w = sent_tokenize(pattern)
        
        words.extend(w)
    
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            c = intent['tag']#.encode("utf-8")
            classes.append(c)
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
#print("words",words)
classes = sorted(list(set(classes)))

print (len(documents), "documents",documents)
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

(27, 'documents', [([u'Hi'], u'greeting'), ([u'How', u'are', u'you'], u'greeting'), ([u'Is', u'anyone', u'there', u'?'], u'greeting'), ([u'Hello'], u'greeting'), ([u'Good', u'day'], u'greeting'), ([u'I', u'would', u'like', u'to', u'diagnose', u'my', u'skin', u'rash'], u'itchy'), ([u'i', u'have', u'a', u'rash'], u'itchy'), ([u'yes', u'it', u'is', u'itching'], u'itchy_resp'), ([u'yeah', u'itches', u'sometimes', u'rarely'], u'itchy_resp'), ([u'not', u'itchy'], u'itchy_resp1'), ([u'no', u'it', u'is', u'not', u'itching'], u'itchy_resp1'), ([u'it', u'does', u'not', u'itch'], u'itchy_resp1'), ([u'yes'], u'exposed_1'), ([u'no'], u'exposed_2'), ([u'1'], u'exposed_resp_1'), ([u'2'], u'exposed_resp_2'), ([u'3'], u'exposed_resp_2'), ([u'4'], u'exposed_resp_2'), ([u'no', u'it', u'is', u'not', u'spreading'], u'spread_resp'), ([u'yes', u'it', u'is', u'spreading'], u'spread_resp'), ([u'yes', u'it', u'spreads'], u'spread_resp'), ([u'Bye'], u'bye'), ([u'See', u'you', u'later'], u'bye'), ([u'Goodbye'], u

In [5]:
training = []
output = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    print(pattern_words)
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    print(bag)

    output_row = list(output_empty)
    print(output_row)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

[u'hi']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[u'how', u'ar', u'you']
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[u'is', u'anyon', u'ther', u'?']
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[u'hello']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[u'good', u'day']
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[u'i', u'would', u'lik', u'to', u'diagnos', u'my', u'skin', u'rash']
[0, 0, 0, 0, 0, 0

In [6]:
tf.reset_default_graph()
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.00940 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.00940 - acc: 1.0000 -- iter: 24/27
Training Step: 4000  | total loss: 0.00923 | time: 0.013s
| Adam | epoch: 1000 | loss: 0.00923 - acc: 1.0000 -- iter: 27/27
--
INFO:tensorflow:/Users/sindhu/Documents/sindhu docs/SJSU/Spring 2018/CMPE295B/tensorflow/chatbot_notebooks/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [7]:
def clean_up_sentence(sentence):
    print(sentence)
    #s = sentence.encode("utf-8")
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    print(sentence_words)
    return sentence_words

def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [8]:
p = bow("hi", words)
print (p)
print (classes)

hi
['hi']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[u'bye', u'exposed_1', u'exposed_2', u'exposed_resp_1', u'exposed_resp_2', u'greeting', u'itchy', u'itchy_resp', u'itchy_resp1', u'spread_resp', u'thank_you']


In [9]:
print(model.predict([p]))


[[  1.85269954e-07   2.93060695e-03   1.43603215e-13   5.73522598e-08
    2.28690193e-03   9.92005646e-01   1.48497700e-14   3.51437484e-05
    1.42612206e-08   2.74141016e-03   2.73119439e-12]]


In [10]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [11]:
ERROR_THRESHOLD = 0.25
def classify(sentence):
    results = model.predict([bow(sentence, words)])[0]
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    print(random.choice(i['responses']))
                    return random.choice(i['responses']).encode("utf-8")

            results.pop(0)

In [12]:
response('hi')

hi
['hi']
Hello, thanks for visiting. What can I do for you?


'Hello, thanks for visiting. What can I do for you?'

In [13]:
response('i have a rash')

i have a rash
['i', 'hav', 'a', 'rash']
is the rash itching?


'is the rash itching?'

In [14]:
response('yes it is itching')

yes it is itching
['ye', 'it', 'is', 'itch']
Were you exposed to sun?


'Were you exposed to sun?'

In [15]:
response('yes')

yes
['ye']
Which part of the body is the rash?
Choose an option
(1).Head&Neck
(2).Legs
(3).Back
(4).Others


'Which part of the body is the rash?\nChoose an option\n(1).Head&Neck\n(2).Legs\n(3).Back\n(4).Others'

In [ ]:
response('1')